# Simple Linear Programming (LP) Example with PuLP

cuOpt is NVIDIA's GPU accelerated solver that delivers massive speedups for real-world LP, MIP, and VRP workloads.

cuOpt seemlessly integrates with modeling languages. You can drop cuOpt into existing models built with PuLP and AMPL, with minimal refactoring. Let's take a look at an example solving a simple LP problem with cuOpt.

To run this in Google Colab, download the notebook and upload it to Google Colab. Make sure you are running this on a T4 GPU.

If you are running this in the cuOpt container, you are good to go!

This example is adapted from CVXPY. You can look at the original example [here](https://www.cvxpy.org/examples/basic/linear_program.html)

## 1. Install Dependencies

To make sure we are good to go, let's install PuLP and cuOpt.

__[PuLP](https://coin-or.github.io/pulp/)__ is a popular linear and mixed integer programming modeler written in Python.


If you are running this notebook in Google Colab, or elsewhere outside the container where cuOpt is not yet installed, uncomment the pip install command to install cuOpt.


In [ ]:
!pip install pulp==3.2.0

In [ ]:
# # Enable this in case you are running this in google colab or such places where cuOpt is not yet installed

#!pip install --upgrade --extra-index-url=https://pypi.nvidia.com cuopt-cu12

## 2. Problem Setup

This optimization problem defines a randomly generated linear program (LP) with 10 decision variables and 15 inequality constraints. The objective is to minimize a linear function of the variables, defined by a vector c, subject to linear inequality constraints of the form Ax≤b, where the matrix A and vector b are constructed to ensure feasibility using random values.


In [ ]:
# Import packages.
from pulp import *
import numpy as np

# Generate a random non-trivial linear program.
m = 15
n = 10
np.random.seed(1)
s0 = np.random.randn(m)
lamb0 = np.maximum(-s0, 0)
s0 = np.maximum(s0, 0)
x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A @ x0 + s0
c = -A.T @ lamb0

# Define and solve the Pulp problem
prob = LpProblem("LP_example", LpMinimize)
x = [LpVariable(f"x{i}", lowBound=None) for i in range(n)]
prob += lpSum([c[i] * x[i] for i in range(n)]), "Objective"

for i in range(m):
    prob += lpSum([A[i, j] * x[j] for j in range(n)]) <= b[i], f"Constraint_{i}"

## 3. Problem Solution
The problem is solved using the CUOPT solver, and the solution yields both the minimum objective value and the corresponding optimal variable values x. This setup demonstrates how to programmatically generate and solve a non-trivial LP using PuLP and NumPy.

In [ ]:
status = prob.solve(CUOPT(msg=0))

# Print results
print("\nThe optimal value is", value(prob.objective))
x_vals = np.array([x[i].varValue for i in range(n)])
np.set_printoptions(precision=8, suppress=True)
print("A solution x is")
print(x_vals)